In [4]:
import pandas as pd
import numpy as np
import sys

In [5]:
import utils
from DataLoader import DataLoader as DL
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

In [6]:
import statsmodels.tsa.api as sta
import statsmodels.tsa.stattools as stattools
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import statsmodels.stats.stattools as stools

import scipy.stats as stats

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LassoCV
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [8]:
#deleted the four ['saleq_mean', 'saleq_std', 'ltmibq_mean', 'ltmibq_std'] in the dataloader file
loc = '/Users/harshbaheti/Desktop/untitled folder/practicum'
d = DL(loc)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [9]:
full_data = pd.read_csv('{loc}/merged_table_full.csv'.format(loc=loc))

In [10]:
full_data.head()

,Unnamed: 0,gvkey,datadate,fyearq,datafqtr,acomincq,acoq,actq,apq,atq,...,mcap_date,breakpt,i1,i2,i3,i4,i5,i6,i7,i8
0,0,1010,1969-09-30,1969,1969Q2,NaN,NaN,NaN,NaN,NaN,...,1969-09-30,49380.0,0,0,1,0,0,0,0,0
1,1,1010,1969-12-31,1969,1969Q3,NaN,NaN,NaN,NaN,NaN,...,1969-12-31,47620.0,0,0,1,0,0,0,0,0
2,2,1010,1970-03-31,1969,1969Q4,NaN,NaN,NaN,NaN,NaN,...,1970-03-31,44150.0,0,0,1,0,0,0,0,0
3,3,1010,1970-06-30,1970,1970Q1,NaN,NaN,NaN,NaN,NaN,...,1970-06-30,29920.0,0,0,1,0,0,0,0,0
4,4,1010,1970-09-30,1970,1970Q2,NaN,NaN,NaN,NaN,NaN,...,1970-09-30,37070.0,0,0,1,0,0,0,0,0


In [11]:
fulldata = full_data[full_data.columns[1:]]


In [12]:
def norm_rank(kk):
    ## for every row, rank from small to big, then minus 0.5, 
    ## then divide number of data in row
    ## ppf: inverse function of cdf
    ## - 0.5 make sure no 1 or 0 in the ppf
    tmp = stats.norm.ppf((kk.rank(1)-.5).div(kk.count(1), 0))
    ## set index and columns
    kk = pd.DataFrame(tmp, index=kk.index, columns=kk.columns)
    ## normalization
    kk = kk.div(kk.std(1), 0)
    return kk

def chi_rank(kk):
    ## for every row, rank from small to big, then minus 0.5, 
    ## then divide number of data in row
    ## ppf: inverse function of cdf
    ## - 0.5 make sure no 1 or 0 in the ppf
    tmp = stats.chi2.ppf((kk.rank(1)-.5).div(kk.count(1), 0), 1)
    ## set index and columns
    kk = pd.DataFrame(tmp, index=kk.index, columns=kk.columns)
    ## normalization
    kk = kk.sub(kk.mean(1),0).div(kk.std(1), 0)
    return kk

def get_cross_section(df,col_name):
    tt = df.stack().reset_index().rename(columns={0: col_name})
    return tt

In [13]:
import tqdm

In [14]:

all_vars = {}
for i in tqdm.tqdm(range(0,len(d.cols_q))):
    all_vars[d.cols_q[i]] = d.get_time_series(fulldata, d.cols_q[i], icol='datadate')

100%|██████████| 60/60 [00:38<00:00,  1.49it/s]


In [15]:
mcap = d.get_time_series(fulldata, 'mcap')
ret = d.get_time_series(fulldata, 'RET')

## if data's mcap is bigger than breakpoint
fulldata.loc[:, 'in_universe']=[np.nan if pd.isnull(m) or pd.isnull(b) or m < b else 1 
                                      for m,b in zip(fulldata.mcap.values, fulldata.breakpt.values)]

In [16]:










start = '1977'
end = '2019'

for x in tqdm.tqdm(range(1,9)):
    industry = 'i'+str(x)
    #get industry universe
    in_universe = d.get_time_series(fulldata[fulldata[industry]==1], 'in_universe')
    in_univ = in_universe[start:end].fillna(method='ffill', limit=3)
    
    cols = in_univ.loc[:, in_univ.count()>0].columns
    len(cols), len(in_univ.columns)
    
    # book value growth
    equity = (all_vars['atq'].loc[start:end,cols] - all_vars['ltq'].loc[start:end,cols])

    new_metric = np.log(1+equity.diff()/all_vars['atq'].shift(1))
    new_metric_mean = new_metric.rolling(20).mean()
    new_metric_std = new_metric.rolling(20).std()

    equity_chg = np.log(equity).diff().rolling(20).mean()
    equity_chg_std = np.log(equity).diff().rolling(20).std()

    y_equity_chg = norm_rank(new_metric_mean * in_univ).stack().reset_index().rename(columns={0: 'book_value growth'})
    
    # book to market
    b2m = np.log(equity / mcap * 1000)
    y_b2m = norm_rank(b2m).stack().reset_index().rename(columns={0: 'book to market'})
    
    def remove_extreme_values(df, limit=8):
        df_med = df.quantile(0.5, 1)
        df_med_dev = np.abs(df.sub(df_med, 0)).quantile(0.5, 1)

        up = df.sub(df_med.add(df_med_dev * limit, 0), 0)
        dn = df.sub(df_med.sub(df_med_dev * limit, 0), 0)
    
        return df[(up<0) & (dn>0)]

    def fill_missing_w_medians(df, univ):
        dfu = df * univ
        dfu_med = dfu.quantile(0.5, 1)
    
        tmp = pd.DataFrame(index=df.index, columns=df.columns).fillna(0)
        tmp = tmp.add(dfu_med, 0)
        tmp[~pd.isnull(dfu)] = df[~pd.isnull(dfu)]
        tmp[pd.isnull(univ)] = np.nan
    
        return tmp

    #NEW
    regressors = []

    for c in d.cols_q:
    # 5 year mean
        if c != 'atq': 
            tt = (all_vars[c] / all_vars['atq']).loc[start:end,cols]
        else:    
            tt = np.log(all_vars[c].loc[start:end,cols]).diff()
    
        tt = remove_extreme_values(tt * in_univ)
    
    # too many missing values
        if ( tt.stack().count() / in_univ.stack().count()) < 0.2:
            continue
    
    # too many zero values
        t1 = tt.stack()
        if (t1[t1==0].count()/(t1.count())) > 0.8:
            continue
        
    # too many missing values
        if ( tt.stack().count() / in_univ.stack().count()) < 0.8:
            continue

        tt = fill_missing_w_medians(tt, in_univ)
        
        all_vars['x_mean_' + c] = norm_rank(tt.rolling(20, min_periods=16).mean()).stack().reset_index().rename(columns={0: c+'_mean'})
        regressors.append('x_mean_' + c)
    
        # 5 year std
        all_vars['x_std_' + c] = chi_rank(tt.rolling(20, min_periods=16).std()).stack().reset_index().rename(columns={0: c+'_std'})
        regressors.append('x_std_' + c)
        
        #NEW
    reg_panel = pd.merge(y_equity_chg.shift(20), y_b2m, on=['gvkey', 'datadate'], how ='left')

    for i in regressors:
        reg_panel = pd.merge(reg_panel, all_vars[i], on=['gvkey', 'datadate'], how ='left')
        
    reg_panel = reg_panel.dropna()
    
    svd_res = np.linalg.svd(reg_panel[reg_panel.columns[4:]].iloc[:3000])
    svd_res[1]
    
    split = '2008-01-01'

    X_train = reg_panel[reg_panel['datadate'] < split].iloc[:, 4:]
    y_train = reg_panel[reg_panel['datadate'] < split]['book_value growth']
    
    X_test = reg_panel[reg_panel['datadate'] >= split].iloc[:, 4:]
    y_test = reg_panel[reg_panel['datadate'] >= split]['book_value growth']
    
    def adj_r2_score(X, y):
        adj_r2 = 1 - (1-lasso.score(X, y))*(len(y)-1)/(len(y)-X.shape[1]-1)
    
        return adj_r2

    # This is a function for finding alpha that will keep 6-7 non-zero coefficients
    #def lasso_tuning(X_train, y_train):
       # for alpha in list(range(0, 100, 1)):
        #    alpha = alpha/100
         #   lasso = Lasso(alpha=alpha)
          #  lasso.fit(X_train, y_train)
           # lasso_coef = lasso.coef_
            #nonzero = np.nonzero(lasso_coef)[0]
            #if len(nonzero) in range (6,9):
             #   return alpha
        #elif len(nonzero) == 8:
            #return alpha
            
    # This is a function for finding alpha that will keep 6-7 non-zero coefficients
    def lasso_tuning(X_train, y_train):
        r2 = 0
        best_alpha = 999
        for alpha in list(range(0, 100, 1)):
            tmp = alpha/100
            lasso = Lasso(alpha=tmp)
            lasso.fit(X_train, y_train)
            adj_r2 = 1 - (1-lasso.score(X_test, y_test))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
            lasso_coef = lasso.coef_
            nonzero = np.nonzero(lasso_coef)[0]
            if len(nonzero) < 6:
                return best_alpha
            if len(nonzero) in range(6,9):
                if adj_r2 > r2:
                    r2 = adj_r2
                    best_alpha = tmp
            
    new_alpha = lasso_tuning(X_train, y_train)
    
    lasso = Lasso(alpha=new_alpha)

    lasso.fit(X_train, y_train)

    # Compute and print the non-zero coefficients
    lasso_coef = lasso.coef_
    coefficient = list(zip(X_train.columns,lasso.coef_))
    for i in range(len(lasso_coef)):
        if lasso_coef[i] != 0:
            print('Coefficient ', X_train.columns[i],' =',str(lasso.coef_[i]))
            
    
    
    print('R^2 train: %.3f, test: %.3f' % (
        lasso.score(X_train, y_train),
        lasso.score(X_test, y_test)))
    print('Adjusted R^2 train: %.3f, test: %.3f' % (
        adj_r2_score(X_train, y_train),
        adj_r2_score(X_test, y_test)))
    
    
    individual_coeff_matrix = pd.DataFrame(coefficient, columns = ['X', 'i_'+str(x)]) 
    
    if(x==1):
        y = []
        z = ['acomincq','acoq','actq','apq','atq','capr1q','capr2q','capr3q','capsq','cheq','chq','cshtrq','cstkeq','cstkq','dlcq','dlttq','doq','dpq','dvpq',
                         'epsfiq','esubq','fcaq','gdwlq','icaptq','intanq',
                         'invtq','ivaeqq','ivaoq','lctq','ltq','mibtq',
                         'miiq','niq','nopiq','oiadpq','ppentq','pstknq','pstkrq',
                         'rectq','req','revtq','rllq','seqoq','seqq','spiq',
                         'teqq','tieq','tiiq','tstkq','txdiq','txditcq','txpq','txtq',
                        'uniamiq','utemq','wcapq','xintq','xiq','xoprq','xrdq']
        for i in (z):
            y = y + [i+"_mean"]
            y = y + [i+"_std"]
        
        w = []
        
            
        
            
        final_coeff_matrix = pd.DataFrame(np.array(y), columns= list("X"))
            
    f= [lasso.score(X_train, y_train),lasso.score(X_test, y_test)]
    w.extend(f)
    final_coeff_matrix = pd.merge(final_coeff_matrix, individual_coeff_matrix, on = ['X','X'], how = 'left')
    
dfi = pd.DataFrame(np.array(w).reshape(8,2),columns=['R2_TRAIN','R2_TEST'])
dfi.index = ['i1','i2','i3','i4','i5','i6','i7','i8']
        
    
    
    
    

  0%|          | 0/8 [00:00<?, ?it/s]/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in log
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in log
/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1904: RuntimeWarning: invalid value encountered in greater
  cond1 = (0 < q) & (q < 1)
/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1904: RuntimeWarning: invalid value encountered in less
  cond1 = (0 < q) & (q < 1)
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in log
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:137: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:477: UserWarning: Coordinate d

Coefficient  atq_mean  = 0.47734629592187916
Coefficient  capsq_mean  = 0.09646880198973436
Coefficient  cshtrq_std  = 0.004327185374327996
Coefficient  icaptq_mean  = 0.039931146558329515
Coefficient  ltq_mean  = -0.07732890772764843
Coefficient  niq_mean  = 0.06706335279826292
Coefficient  oiadpq_std  = 0.03217077828931741
R^2 train: 0.488, test: 0.299
Adjusted R^2 train: 0.482, test: 0.275


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:137: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:477: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
 25%|██▌       | 2/8 [00:12<00:41,  6.94s/it]

Coefficient  atq_mean  = 0.33581963530256276
Coefficient  capsq_mean  = 0.048128557946455784
Coefficient  capsq_std  = 0.002599072770002295
Coefficient  cshtrq_mean  = 0.038960695967125224
Coefficient  niq_mean  = 0.306086253287886
Coefficient  oiadpq_mean  = 0.05481590785726964
R^2 train: 0.523, test: 0.472
Adjusted R^2 train: 0.504, test: 0.388


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in log
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: RuntimeWarning: divide by zero encountered in log
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: RuntimeWarning: divide by zero encountered in log
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: RuntimeWarning: divide by zero encountered in log
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:137: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:477: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want 

...

Coefficient  atq_mean  = 0.5181909525377181
Coefficient  cheq_mean  = 0.0007735161184207567
Coefficient  cheq_std  = 0.0275614774151393
Coefficient  cshtrq_mean  = 0.03358052522270263
Coefficient  cshtrq_std  = 0.04996540601787734
Coefficient  niq_mean  = 0.007967725906824437
Coefficient  oiadpq_mean  = 0.040870787090604904
Coefficient  txtq_mean  = 0.025316493527706003
R^2 train: 0.566, test: 0.413
Adjusted R^2 train: 0.559, test: 0.373


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:137: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:477: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
 88%|████████▊ | 7/8 [01:28<00:10, 10.61s/it]

Coefficient  atq_mean  = 0.3527691511381604
Coefficient  cheq_std  = 0.024373329310972032
Coefficient  epsfiq_mean  = 0.12930656827806958
Coefficient  epsfiq_std  = 0.027421838615435786
Coefficient  icaptq_mean  = 0.12535634638650772
Coefficient  nopiq_std  = 0.029652933765155223
Coefficient  rectq_mean  = -0.07111252194088685
Coefficient  txtq_mean  = 0.12432151864363009
R^2 train: 0.382, test: 0.361
Adjusted R^2 train: 0.382, test: 0.359


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:137: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:477: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
100%|██████████| 8/8 [01:44<00:00, 12.21s/it]

Coefficient  atq_mean  = 0.5015274783919087
Coefficient  capsq_mean  = 0.01220041372595042
Coefficient  cheq_std  = 0.0701493899974232
Coefficient  cshtrq_mean  = 0.05759992656154326
Coefficient  epsfiq_mean  = 0.006759685368857197
Coefficient  icaptq_mean  = 0.03342080605376818
Coefficient  niq_mean  = 0.01038134452846075
Coefficient  seqq_mean  = 0.12282775156376727
R^2 train: 0.551, test: 0.510
Adjusted R^2 train: 0.549, test: 0.503


In [17]:
df_all = final_coeff_matrix.set_index('X')



In [18]:
X_train.shape, X_test.shape

((10393, 48), (3673, 48))

In [19]:
df_all = df_all.fillna(0)
df_all.columns = ['mining','construction','manufacturing','TCEGS','wholesale','retail','Finance','service']

for y in df_all.index:
    n = 0
    for x in range(0,8):
        if df_all.loc[y][x] == 0:
            n = n+1
        if n == 7:
            df_all.drop(y,inplace = True)
            break

In [20]:
print("train: "+start+"-"+ split[:4]+ " test: "+ split[:4] +"-"+ end)
print(dfi)
print(df_all)



train: 1977-2009 test: 2009-2015
    R2_TRAIN   R2_TEST
i1  0.487811  0.298684
i2  0.523015  0.472301
i3  0.560890  0.479729
i4  0.455144  0.316341
i5  0.611114  0.395370
i6  0.566266  0.413318
i7  0.382325  0.361086
i8  0.551319  0.509701
               mining  construction  manufacturing     TCEGS  wholesale  \
X                                                                         
atq_mean     0.477346      0.335820       0.487892  0.440233   0.461085   
capsq_mean   0.096469      0.048129       0.000000  0.000000   0.000000   
capsq_std    0.000000      0.002599       0.071101  0.034849   0.000000   
cheq_std     0.000000     -0.000000       0.043659  0.000000   0.000000   
cshtrq_mean  0.000000      0.038961       0.000000  0.000000   0.026660   
cshtrq_std   0.004327      0.000000       0.015768  0.019305   0.023610   
epsfiq_mean  0.000000      0.000000       0.000000  0.000000   0.023375   
icaptq_mean  0.039931      0.000000       0.099013  0.028818   0.034298   
ltq_mean  